In [1]:
import numpy as np
import pandas as pd

## Load Data

### Load genre_gn_all.json

In [2]:
genre_gn_all = pd.read_json('./data/genre_gn_all.json', typ = 'series', encoding = 'UTF-8')
genre_gn_all = pd.DataFrame(genre_gn_all, columns = ['gnr_name']).reset_index().rename(columns = {'index' : 'gnr_code'})
genre_gn_all.head()

,gnr_code,gnr_name
0,GN0100,발라드
1,GN0101,세부장르전체
2,GN0102,'80
3,GN0103,'90
4,GN0104,'00


### Load song_meta.json

In [3]:
song_meta = pd.read_json('./data/song_meta.json', orient='values', encoding = 'UTF-8')

In [4]:
song_meta = song_meta.set_index('id')
song_meta.head()

,album_id,album_name,artist_id_basket,artist_name_basket,issue_date,song_gn_dtl_gnr_basket,song_gn_gnr_basket,song_name
id,,,,,,,,
0,2255639,불후의 명곡 - 7080 추억의 얄개시대 팝송베스트,[2727],[Various Artists],20140512,[GN0901],[GN0900],Feelings
1,376431,"Bach : Partitas Nos. 2, 3 & 4",[29966],[Murray Perahia],20080421,"[GN1601, GN1606]",[GN1600],"Bach : Partita No. 4 In D Major, BWV 828 - II...."
2,4698747,Hit,[3361],[Peter Gabriel],20180518,[GN0901],[GN0900],Solsbury Hill (Remastered 2002)
3,2644882,Feeling Right (Everything Is Nice) (Feat. Popc...,[838543],[Matoma],20151016,"[GN1102, GN1101]",[GN1100],Feeling Right (Everything Is Nice) (Feat. Popc...
4,2008470,그남자 그여자,[560160],[Jude Law],20110824,"[GN1802, GN1801]",[GN1800],그남자 그여자


### Load train.json

In [5]:
train_data = pd.read_json('./data/train.json', encoding = 'UTF-8')
train_data.head()

,id,like_cnt,plylst_title,songs,tags,updt_date
0,61281,71,여행같은 음악,"[525514, 129701, 383374, 562083, 297861, 13954...",[락],2013-12-19 18:36:19.000
1,10532,1,요즘 너 말야,"[432406, 675945, 497066, 120377, 389529, 24427...","[추억, 회상]",2014-12-02 16:19:42.000
2,76951,17,"편하게, 잔잔하게 들을 수 있는 곡.-","[83116, 276692, 166267, 186301, 354465, 256598...","[까페, 잔잔한]",2017-08-28 07:09:34.000
3,147456,33,크리스마스 분위기에 흠뻑 취하고 싶을때,"[394031, 195524, 540149, 287984, 440773, 10033...","[연말, 눈오는날, 캐럴, 분위기, 따듯한, 크리스마스캐럴, 겨울노래, 크리스마스,...",2019-12-05 15:15:18.000
4,27616,9,추억의 노래 ㅋ,"[159327, 553610, 5130, 645103, 294435, 100657,...",[댄스],2011-10-25 13:54:56.000


- playlist id - songs 맵핑

In [6]:
# 플레이리스트 아이디(id)와 수록곡(songs) 추출
plylst_song_map = train_data[['id', 'songs']]

# unnest songs
plylst_song_map_unnest = np.dstack(
    (
        np.repeat(plylst_song_map.id.values, list(map(len, plylst_song_map.songs))), 
        np.concatenate(plylst_song_map.songs.values)
    )
)

# unnested 데이터프레임 생성 : plylst_song_map
plylst_song_map = pd.DataFrame(data = plylst_song_map_unnest[0], columns = plylst_song_map.columns)
plylst_song_map['id'] = plylst_song_map['id'].astype(str)
plylst_song_map['songs'] = plylst_song_map['songs'].astype(str)

# unnest 객체 제거
del plylst_song_map_unnest

In [7]:
plylst_song_map.head()

,id,songs
0,61281,525514
1,61281,129701
2,61281,383374
3,61281,562083
4,61281,297861


In [8]:
# 1. 곡 별 수록된 플레이리스트 개수 count 테이블 생성 : song_plylst_cnt
song_plylst_cnt = plylst_song_map.groupby('songs').id.nunique().reset_index(name = 'mapping_plylst_cnt')

# 2. 단일/중복 수록 구분 : 곡 별 수록된 플레이리스트가 한 개면 '단일 수록', 두 개 이상이면 '중복 수록'
song_plylst_cnt = song_plylst_cnt.assign(
    mapping_plylst_cnt_category = pd.cut(song_plylst_cnt['mapping_plylst_cnt'], [0, 9, np.inf], labels = ['10회 미만', '10회 이상'])
)

In [9]:
# 10회 이상 나오는 곡의 수
len(song_plylst_cnt[song_plylst_cnt.mapping_plylst_cnt_category == '10회 이상'].songs)

81221

In [10]:
plylst_song_map_filter = plylst_song_map[plylst_song_map.songs.isin(song_plylst_cnt[song_plylst_cnt.mapping_plylst_cnt_category == '10회 이상'].songs)]

In [11]:
print(plylst_song_map.shape)
print(plylst_song_map_filter.shape)

(5285871, 2)
(4123576, 2)


In [12]:
train_data = train_data.set_index('id')

# tag list를 하나의 리스트로 합쳐주는 함수
def merge_list(lst):
    out = set()
    for sublst in lst:
        out = out.union(set(sublst))
    return list(out)

merge_list(train_data.loc[[61281, 10532, 147456], :].loc[:, 'tags'].values)

['회상',
 '따듯한',
 '캐럴',
 '크리스마스송',
 '크리스마스캐럴',
 '연말',
 '겨울노래',
 '락',
 '눈오는날',
 '추억',
 '겨울왕국',
 '크리스마스',
 '분위기']

**아래 코드 8분 정도 소요됨**

In [13]:
song_tag_map = plylst_song_map_filter.groupby('songs').apply(lambda x : merge_list(train_data.loc[x['id'].values.astype(np.int32), :].tags))
song_tag_map.head()

songs
10        [회상, 겨울, 카페, 연주곡, 로파이힙합, Chill, 감성, Lofihiphop...
1000      [로멘틱, 겨울, 뉴에이지, 팝, 카페, 연주곡, 배경음악, 연말, Chill, 월...
100011    [겨울, 회상, 팝, 카페, 일요일, 위로, 연말, 산책, 신나는, 휴식, 기분전환...
100014    [뉴에이지, 몽환, 연주곡, 힐링, 산책, 휴식, 기분전환, 매장음악, 새벽, 까페...
100020    [요가음악, 요가, 뉴에이지, 카페, 공부, 연주곡, 위로, 휴식, 북카페, 편안한...
dtype: object

In [14]:
song_tag_map_frame = pd.DataFrame(song_tag_map)
song_tag_map_frame = song_tag_map_frame.rename(columns = {0 : 'tags'})
song_tag_map_frame = song_tag_map_frame.reset_index()
song_tag_map_frame.head()

,songs,tags
0,10,"[회상, 겨울, 카페, 연주곡, 로파이힙합, Chill, 감성, Lofihiphop..."
1,1000,"[로멘틱, 겨울, 뉴에이지, 팝, 카페, 연주곡, 배경음악, 연말, Chill, 월..."
2,100011,"[겨울, 회상, 팝, 카페, 일요일, 위로, 연말, 산책, 신나는, 휴식, 기분전환..."
3,100014,"[뉴에이지, 몽환, 연주곡, 힐링, 산책, 휴식, 기분전환, 매장음악, 새벽, 까페..."
4,100020,"[요가음악, 요가, 뉴에이지, 카페, 공부, 연주곡, 위로, 휴식, 북카페, 편안한..."


In [15]:
# 전체 태그 set 만들기
tags = set({})
for subtags in song_tag_map_frame.tags.values:
    tags = tags.union(set(subtags))
len(tags) # 전체 태그 수

27619

In [16]:
# 태그 set -> dictionary
tags_dict = {list(tags)[i]: i for i in range(len(tags))}

In [17]:
tags_dict['휴식']

19827

In [18]:
# 태그 -> id
song_tag_map_frame.tags = song_tag_map_frame.tags.apply(lambda x : [tags_dict[t] for t in x])

In [19]:
song_tag_map_frame.head()

,songs,tags
0,10,"[19881, 22680, 13581, 7424, 9570, 19666, 16886..."
1,1000,"[18650, 22680, 16764, 12907, 13581, 7424, 1142..."
2,100011,"[22680, 19881, 12907, 13581, 26209, 20145, 319..."
3,100014,"[16764, 10989, 7424, 17586, 1344, 19827, 21640..."
4,100020,"[2114, 14320, 16764, 13581, 6123, 7424, 20145,..."


Save song_tag_map_frame to csv file

In [20]:
song_tag_map_frame.to_csv(path_or_buf='song_tag_id_map.csv', encoding='UTF-8')

In [21]:
song_tag_map_frame.to_pickle(path = '../song_tag_id_map.pkl')

### Load val.json

In [22]:
val_data = pd.read_json('./data/val.json', encoding = 'UTF-8')
val_data.head()

,id,like_cnt,plylst_title,songs,tags,updt_date
0,118598,1675,,"[373313, 151080, 275346, 696876, 165237, 52593...",[],2019-05-27 14:14:33.000
1,131447,1,앨리스테이블,[],[],2014-07-16 15:24:24.000
2,51464,62,,"[529437, 516103, 360067, 705713, 226062, 37089...",[],2008-06-21 23:26:22.000
3,45144,20,,"[589668, 21711, 570151, 320043, 13930, 599327,...",[],2017-10-30 18:15:43.000
4,79929,20,,"[672718, 121924, 102694, 683657, 201558, 38511...",[],2017-02-07 11:40:42.000


In [27]:
val_data.shape

(23015, 6)

In [26]:
val_data[(val_data.songs.apply(len) == 0) & (val_data.tags.apply(len) == 0)]

,id,like_cnt,plylst_title,songs,tags,updt_date
1,131447,1,앨리스테이블,[],[],2014-07-16 15:24:24.000
9,142007,0,기분 좋은 재즈와 함께 만드는 달달한 하루,[],[],2015-06-22 09:11:02.000
35,65114,6,"■■■■ 사랑,그리고이별 ■■■■",[],[],2010-10-27 10:34:34.000
57,87700,6,마쉬멜로우같은 멜로우한 음악,[],[],2016-01-14 10:19:30.000
71,35271,10,공부와 독서를 위한 #Newage,[],[],2020-01-17 15:46:20.000
101,85920,0,새벽에 감성 돋는 노래,[],[],2019-09-05 20:53:20.000
115,122041,0,"인디, OST 음악을 만나다 50",[],[],2019-03-27 15:28:45.000
135,62695,20,너 없이 살수가 없어(그리움),[],[],2017-09-16 19:16:57.000
167,116425,2548,귀를 달달하게 만들어주는 남녀 듀엣곡,[],[],2016-04-29 12:26:44.000
199,67281,18,★기분전환 K-POP 뮤직★,[],[],2013-06-18 09:26:02.000
